<a href="https://colab.research.google.com/github/giovannibaratta/GTSRB/blob/master/GroupModelHosted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Import e definizioni (Versione 5) { vertical-output: true, display-mode: "form" }
#@markdown Selezionare se si vuole caricare una versione specifica di tensorflow
forceVersion = False #@param {type:"boolean"} 
tfVersion = "1.13.1" #@param ["1.12.2", "1.13.1", "1.14.0rc1", "1.14.0", "2.0.0b1", "PRINT_AV_VERSION"] {allow-input: true}
#@markdown Selezionare se si è collegati ad un runtime locale (non quello di colab)
localRuntime = False #@param{type:"boolean"}
#@markdown Selezionare se si intende utilizzare le TPU come acceleratore (solo se il runtime è quello di colab)
useTPU = False #@param {type:"boolean"}

useTPU = False if localRuntime else useTPU

if forceVersion == True:
  if useTPU == True:
    !pip install -q tensorflow=={tfVersion}
  else:
    !pip install -q tensorflow-gpu=={tfVersion}

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pprint
import math
import random as rn
#data aug
from tensorflow import keras 
# gestione directory
import shutil
import sys
# colorare output
!pip install -q colorama
from colorama import Style as ColoramaStyle
import re
from datetime import datetime

currentPath = !pwd
#@markdown Path da utilizzare nel caso di runtime remoto
colabPath = '/gdrive/My Drive/DatasetSegnaliStradali/GTSRB/' #@param{type:'string'} 
#@markdown Path da utilizzare nel caso di runtime locale
localPath =  "/Desktop/DL/" #@param{type:'string'} 
completeLocalPath = currentPath[0] + localPath
#@markdown Cartella da utilizzare all'interno del path per l'output dei risultati (la cartella deve essere già presente)
phase = "trainingPhaseGroupModel" #@param ["trainingPhase1", "trainingPhaseCNN", "trainingPhaseResNet", "trainingPhaseInception", "trainingPhaseGroupModel"] {allow-input: true}
rootPath = completeLocalPath if localRuntime else colabPath

#@markdown Dimensioni delle immagini di training, validation e test
width = 48#@param{type:'integer'}
height = 48#@param{type:'integer'}
tmpPath = rootPath + 'tmp/'
testDir = rootPath + 'data/test' + str(width) + 'x' + str(height) + '/'

print('Versione TF : ', ColoramaStyle.BRIGHT, tf.__version__, ColoramaStyle.RESET_ALL,sep="")

if useTPU == False:
  if 'COLAB_TPU_ADDR' in os.environ:
    raise RuntimeError("Hai selezionato il runtime TPU. Cambia l'impostazioni del notebook")
  print("Acceleratore : GPU")
  print("Verifica device :",tf.test.gpu_device_name())
  tpu_address = ''
else:
  if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
  else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print("Acceleratore : TPU")
    print ('Verifica device : ', tpu_address)

if not localRuntime:
  from google.colab import drive
  from google.colab import files
  drive.mount('/gdrive')
    
#import script da gdrive
scriptPath = rootPath + "scripts/colab/"
sys.path.append(scriptPath)

from CommonUtils import *
from ModelBuilderUtils import *
from TrainingUtils import *
from TestUtils import *

if tfVersion()['MAJOR'] < 2:
  tf.logging.set_verbosity(tf.logging.ERROR)
resetSeed()

groupInfo = {
    0 : 'prohibitory',
    1 : 'danger',
    2 : 'mandatory',
    3 : 'others',
    4 : 'limitsEnd'
}

labelsInfo = {
    # labelNumber : (group,labelInsideGroup, description)
    0 : (0,0, '20 km/h'),
    1 : (0,1, '30 km/h'),
    2 : (0,2, '50 km/h'),
    3 : (0,3, '60 km/h'),
    4 : (0,4, '70 km/h'),
    5 : (0,5, '80 km/h'),
    6 : (4,0, '80 km/h end'),
    7 : (0,6, '100 km/h'),
    8 : (0,7, '120 km/h'),
    9 : (0,8, 'No overtaking'),
    10 : (0,9, 'No overtaking for tracks'),
    11 : (1,0, 'Crossroad with secondary way'),
    12 : (3,0, 'Main road'),
    13 : (3,1,  'Give way'),
    14 : (3,2, 'Stop'),
    15 : (0,10,  'Road up'),
    16 : (0,11, 'Road up for track'),
    17 : (3,3,  'Brock'),
    18 : (1,1, 'Other dangerous'),
    19 : (1,2, 'Turn left'),
    20 : (1,3, 'Turn right'),
    21 : (1,4, 'Winding road'),
    22 : (1,5, 'Hollow road'),
    23 : (1,6, 'Slippery road'),
    24 : (1,7, 'Narrowing road'),
    25 : (1,8, 'Roadwork'),
    26 : (1,9, 'Traffic light'),
    27 : (1,10, 'Pedestrian'),
    28 : (1,11, 'Children'),
    29 : (1,12, 'Bike'),
    30 : (1,13, 'Snow'),
    31 : (1,14, 'Deer'),
    32 : (4,1, 'End of the limits'),
    33 : (2,0, 'Only right'),
    34 : (2,1, 'Only left'),
    35 : (2,2, 'Only straight'),
    36 : (2,3, 'Only straight and right'),
    37 : (2,4, 'Only straight and left'),
    38 : (2,5, 'Take right'),
    39 : (2,6, 'Take left'),
    40 : (2,7, 'Circle crossroad'),
    41 : (4,2, 'End of overtaking limit'),
    42 : (4,3, 'End of overtaking limit for track')
}

lookUpTable=[
    # gruppo 0
    {
        0:0,
        1:1,
        2:2,
        3:3,
        4:4,
        5:5,
        6:7,
        7:8,
        8:9,
        9:10,
        10:15,
        11:16
    },
    # gruppo 1
    {
        0:11,
        1:18,
        2:19,
        3:20,
        4:21,
        5:22,
        6:23,
        7:24,
        8:25,
        9:26,
        10:27,
        11:28,
        12:29,
        13:30,
        14:31
    },
    # gruppo 2
    {
        0:33,
        1:34,
        2:35,
        3:36,
        4:37,
        5:38,
        6:39,
        7:40
    },
    # gruppo 3
    {
        0:12,
        1:13,
        2:14,
        3:17
    },
    #gruppo 4
    {
        0:6,
        1:32,
        2:41,
        3:42  
    }
]

### Caricamento dati, definizione modello e training per riconoscere il gruppo delle immagini (prohibitory,danger,mandatory,others,limits end)

In [0]:
#@title Caricamento dei dati (versione 5) { vertical-output: true, display-mode: "form" }
if 'trainingData' in locals():
  del trainingData
if 'trainingLabels' in locals():
  del trainingLabels
if 'validationData' in locals():  
  del validationData
if 'validationLabels' in locals():
  del validationLabels

trainingDir = rootPath + 'data/training/'
validationDir = rootPath + 'data/validation/'

#@markdown Deselezionare nel caso si sti allenando il modello a due step, in quel caso le label non sono le originale ma sono il gruppo della classe
loadTrueLabels = True #@param{type:'boolean'}

samplingToLoad = 1#@param{type:'integer'}
#@markdown Seed per la genezione di nuove immagini
randomSeed = 62806#@param{type:'integer'}
#@markdown Sperimentale, non usare
use16Bit = False#@param{type:'boolean'}
#@markdown Sperimentale, non usare
convertToTesnor = False#@param{type:'boolean'}
trainingDir = rootPath + 'data/sampling' + str(samplingToLoad) + '_' + str(width) + 'x' + str(height) + '/training/'
validationDir = rootPath + 'data/sampling' + str(samplingToLoad) + '_' + str(width) + 'x' + str(height) + '/validation/'

#riproducibilità dei training
resetSeed(15 + randomSeed)

#@markdown Selezionare se si vogliono generare delle immagini aggiuntive
useDataAugmentation = True #@param {type:"boolean"}
#@markdown Se selezionato il valore degli slider vengono aggiunti ai valori presenti nel codice
usePerLabelDataAug = True #@param {type:"boolean"}
#@markdown Se selezionato non genera immagini aggiuntive per il validation set
onlyTrainingAug = False #@param {type:"boolean"}

#@markdown Numero di immagini da generare per ogni immagine originale
NUMBER_OF_TRAINING_AUG_GLOBAL = 1 #@param {type:"slider", min:0, max:20, step:1}
NUMBER_OF_VALIDATION_AUG_GLOBAL = 1 #@param {type:"slider", min:0, max:5, step:1}

# da modificare se si vuole alterare la distribuzione delle classi
NUMBER_AUG_BY_LABEL = { 
                0 : ( 4 , 2 ), # group 0  label inside group 0
                1 : ( 1 , 1 ), # group 0  label inside group 1
                2 : ( 1 , 1 ), # group 0  label inside group 2
                3 : ( 1 , 1 ), # group 0  label inside group 3
                4 : ( 1 , 1 ), # group 0  label inside group 4
                5 : ( 1 , 1 ), # group 0  label inside group 5
                6 : ( 3 , 2 ), # group 4  label inside group 0
                7 : ( 1 , 1 ), # group 0  label inside group 6
                8 : ( 1 , 1 ), # group 0  label inside group 7
                9 : ( 1 , 1 ), # group 0  label inside group 8
                10 : ( 1 , 1 ), # group 0  label inside group 9
                11 : ( 1 , 1 ), # group 1  label inside group 0
                12 : ( 1 , 1 ), # group 3  label inside group 0
                13 : ( 1 , 1 ), # group 3  label inside group 1
                14 : ( 2 , 1 ), # group 3  label inside group 2
                15 : ( 2 , 1 ), # group 0  label inside group 10
                16 : ( 3 , 2 ), # group 0  label inside group 11
                17 : ( 1 , 1 ), # group 3  label inside group 3
                18 : ( 1 , 1 ), # group 1  label inside group 1
                19 : ( 4 , 2 ), # group 1  label inside group 2
                20 : ( 3 , 2 ), # group 1  label inside group 3
                21 : ( 3 , 2 ), # group 1  label inside group 4
                22 : ( 3 , 2 ), # group 1  label inside group 5
                23 : ( 3 , 2 ), # group 1  label inside group 6
                24 : ( 3 , 2 ), # group 1  label inside group 7
                25 : ( 1 , 1 ), # group 1  label inside group 8
                26 : ( 3 , 2 ), # group 1  label inside group 9
                27 : ( 3 , 2 ), # group 1  label inside group 10
                28 : ( 3 , 2 ), # group 1  label inside group 11
                29 : ( 3 , 2 ), # group 1  label inside group 12
                30 : ( 4 , 2 ), # group 1  label inside group 13
                31 : ( 2 , 2 ), # group 1  label inside group 14
                32 : ( 3 , 2 ), # group 4  label inside group 1
                33 : ( 3 , 2 ), # group 2  label inside group 0
                34 : ( 5 , 2 ), # group 2  label inside group 1
                35 : ( 1 , 2 ), # group 2  label inside group 2
                36 : ( 2 , 2 ), # group 2  label inside group 3
                37 : ( 2 , 2 ), # group 2  label inside group 4
                38 : ( 1 , 1 ), # group 2  label inside group 5
                39 : ( 3 , 2 ), # group 2  label inside group 6
                40 : ( 3, 2 ), # group 2  label inside group 7
                41 : ( 3 , 2 ), # group 4  label inside group 2
                42 : ( 3 , 2 ), # group 4  label inside group 3
              }

NUMBER_OF_TRAINING_AUG = [0 for i in range(43)]
NUMBER_OF_VALIDATION_AUG = [0 for i in range(43)]

if useDataAugmentation == True:
  for key in labelsInfo.keys():
    NUMBER_OF_TRAINING_AUG[key] = NUMBER_AUG_BY_LABEL[key][0] if usePerLabelDataAug else NUMBER_OF_TRAINING_AUG_GLOBAL
    NUMBER_OF_VALIDATION_AUG[key] = NUMBER_AUG_BY_LABEL[key][1] if usePerLabelDataAug else NUMBER_OF_VALIDATION_AUG_GLOBAL
    if usePerLabelDataAug and onlyTrainingAug:
        NUMBER_OF_VALIDATION_AUG[key] = 0


brightness_range_training = (0.25,0.6)
rotation_range_training = 25
width_shift_range_training = 8
height_shift_range_training = 8
shear_range_training = 15
zoom_range_training =(0.4,1.15)
channel_shift_range_training = 0.13

brightness_range_validation =(0.25,0.7)
rotation_range_validation = 25
width_shift_range_validation = 8 
height_shift_range_validation = 8
shear_range_validation = 0.4
zoom_range_validation = (0.4,1.15)
channel_shift_range_validation = 0.12

#generatore utilizzato per le immagni di training
generatorTraining = keras.preprocessing.image.ImageDataGenerator(
    brightness_range = brightness_range_training,
    rotation_range= rotation_range_training,
    width_shift_range= width_shift_range_training,
    height_shift_range= height_shift_range_training,
    shear_range=shear_range_training,
    zoom_range= zoom_range_training,
    channel_shift_range=channel_shift_range_training)

#generatore utilizzato per le immagini di validation
generatorValidation = keras.preprocessing.image.ImageDataGenerator(
    brightness_range = brightness_range_validation,
    rotation_range=rotation_range_validation,
    width_shift_range=width_shift_range_validation,
    height_shift_range=height_shift_range_validation,
    shear_range=shear_range_validation,
    zoom_range=zoom_range_validation,
    channel_shift_range=channel_shift_range_validation)

# numero di immagini originali disponibili per ogni classe
trainingClassCount = {}
# numero di immagini di training totali
totalTrainingCount = 0

files = list(filter(lambda fn : fn.startswith('resized'), os.listdir(trainingDir)))
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  numFilePath = trainingDir + 'num' + str(classLabel)
  with open(numFilePath, 'r') as file:
    originalNum = int(file.readline())
    trainingClassCount[classLabel] = originalNum 
    totalTrainingCount += originalNum * (NUMBER_OF_TRAINING_AUG[classLabel] + 1)

print("Training images (no pad):", totalTrainingCount)

# se uso le TPU in tf 1.13 il numero di immagini deve essere divisibile per 8
trainingPadEnd = False

if useTPU == True and tfVersion()['MAJOR'] == 1 and tfVersion()['MAJOR'] <= 13:
  if totalTrainingCount % 8 != 0:
    trainingToPad = 8 - (totalTrainingCount % 8)
    totalTrainingCount = totalTrainingCount + (8 - (totalTrainingCount % 8))
    trainingPadEnd = True
  
#preparo gli array che contengono le img di training
trainingData = np.empty((totalTrainingCount, width, height, 3), dtype='float32')
trainingLabels = np.empty((totalTrainingCount), dtype='int32')
trueTrainingLabels = np.empty((totalTrainingCount), dtype='int32')

# numero di immagini originali disponibili per ogni classe
validationClassCount = {}
# numero di immagini di validation totali
totalValidationCount = 0

# recupero il numero di immagini di validation disponibili per ogni classe
files = list(filter(lambda fn : fn.startswith('resized'), os.listdir(trainingDir)))
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  numFilePath = validationDir + 'num' + str(classLabel)
  with open(numFilePath, 'r') as file:
    originalNum = int(file.readline())
    validationClassCount[classLabel] = originalNum
    totalValidationCount += originalNum * (NUMBER_OF_VALIDATION_AUG[classLabel] + 1)

print("Validation images (no pad):", totalValidationCount)

# se uso le TPU in tf 1.13 il numero di immagini deve essere divisibile per 8
validationPadEnd = False 

if useTPU == True and tfVersion()['MAJOR'] == 1 and tfVersion()['MAJOR'] <= 13:
    if totalValidationCount % 8 != 0:
      # se uso le TPU la lunghezza dei dati deve essere divisibile per 8
      validationToPad = 8 - (totalValidationCount % 8)
      totalValidationCount = totalValidationCount + (8 - (totalValidationCount % 8))
      validationPadEnd = True
  
#preparo gli array che contengono le img di validation
validationData = np.empty((totalValidationCount, width, height, 3), dtype='float32')
# contengono il gruppo da 0 a 4
validationLabels = np.empty((totalValidationCount), dtype='int32')
#contengono la label vera da 0 a 42
trueValidationLabels = np.empty((totalValidationCount), dtype='int32')

print('Total', totalValidationCount + totalTrainingCount)
if useDataAugmentation == True:
  print("Data augmentation training", NUMBER_OF_TRAINING_AUG, "\nvalidation", NUMBER_OF_VALIDATION_AUG)
else:
  print("Data augmentation : OFF")

#numero di immagini elaborate  
validationCount = 0
trainingCount = 0
loadedClass = 0

#caricamento dati
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  #carico le img di training originali
  trainingImages = np.memmap(trainingDir + fileName, dtype=np.uint8,
              mode='r', shape=(trainingClassCount[classLabel],width,height,3))

  for img in trainingImages:
    trainingData[trainingCount]  = img/255.0
    trainingLabels[trainingCount] = labelsInfo[classLabel][0] if not loadTrueLabels else classLabel
    trueTrainingLabels[trainingCount] = classLabel
    trainingCount += 1

  #carico le immagini di validation
  validationImages = np.memmap(validationDir + fileName, dtype=np.uint8,
              mode='r', shape=(validationClassCount[classLabel],width,height,3))

  for img in validationImages:
    validationData[validationCount] = img/255.0
    validationLabels[validationCount] = labelsInfo[classLabel][0] if not loadTrueLabels else classLabel
    trueValidationLabels[validationCount] = classLabel
    validationCount += 1
  
# per ogni immagine genero delle nuove immagini con trasformazioni casuali
if useDataAugmentation == True:
  print("Genero le immagini aggiuntive")
  resetSeed(48560  + randomSeed)
  for imageIndex in range(0, trainingCount):
    for augIndex in range(0, NUMBER_OF_TRAINING_AUG[trueTrainingLabels[imageIndex]]):
      trainingData[trainingCount] = generateRandomImage(generatorTraining, trainingData[imageIndex])/255.0
      trainingLabels[trainingCount] = trainingLabels[imageIndex]
      trueTrainingLabels[trainingCount] = trueTrainingLabels[imageIndex]
      trainingCount += 1

  resetSeed(-420360  + randomSeed)
  for imageIndex in range(0, validationCount):
    for augIndex in range(0, NUMBER_OF_VALIDATION_AUG[trueValidationLabels[imageIndex]]):
      validationData[validationCount] = generateRandomImage(generatorValidation, validationData[imageIndex])/255.0
      validationLabels[validationCount] = validationLabels[imageIndex]
      trueValidationLabels[validationCount] = trueValidationLabels[imageIndex]
      validationCount += 1

# Se necessario copio l'ultima immagine per rendere la lunghezza divisibile per 8
if trainingPadEnd == True:
  for index in range(1, trainingToPad + 1):
    trainingData[-index] = trainingData[len(trainingData) - (trainingToPad + 1)]
    trainingLabels[-index] = trainingLabels[len(trainingLabels) - (trainingToPad + 1)]
if validationPadEnd == True:
  for index in range(1, validationToPad + 1):
    validationData[-index] = validationData[len(validationData) - (validationToPad + 1)]
    validationLabels[-index] = validationLabels[len(validationLabels) - (validationToPad + 1)]

print("")

if use16Bit:
    trainingData = tf.convert_to_tensor(trainingData, dtype=tf.bfloat16)
    validationData = tf.convert_to_tensor(validationData, dtype=tf.bfloat16)
else:
    if convertToTesnor:
        tempTrainingData = tf.convert_to_tensor(trainingData[:1000], dtype=tf.float32)
        tempValidationData = tf.convert_to_tensor(validationData[:1000], dtype=tf.float32)

        for index in range((len(trainingData)//1000)-1):
            tempTrainingData = tf.concat([tempTrainingData, tf.convert_to_tensor(trainingData[1000+1000*index:1000+1000*(index+1)], dtype=tf.float32)],0)
        for index in range((len(validationData)//1000)-1):
            tempValidationData = tf.concat([tempValidationData, tf.convert_to_tensor(validationData[1000+1000*index:1000+1000*(index+1)], dtype=tf.float32)],0)
        trainingData = tf.concat([tempTrainingData, tf.convert_to_tensor(trainingData[1000*(len(trainingData)//1000):len(trainingData)], dtype=tf.float32)],0)
        validationData = tf.concat([tempValidationData, tf.convert_to_tensor(validationData[1000*(len(validationData)//1000):len(validationData)], dtype=tf.float32)],0)
print('Training', trainingData.shape) 
print('Validation', validationData.shape)

# statistiche sulla distribuzione delle labels
plt.figure(figsize=(20,5))
maxLabel = max(trainingLabels)+1
print(maxLabel)
if maxLabel > 43:
  raise RuntimeError("C'è un errore grave nel codice, ricontrolla.")
plt.hist(trainingLabels, alpha = 0.5, bins=maxLabel, label = "training")
plt.hist(validationLabels, alpha = 0.5, bins=maxLabel, label = "validation")
plt.xticks(np.arange(0, maxLabel, 1))
plt.xlim(left=0, right=maxLabel)
plt.legend(loc='upper right')
plt.show()

In [0]:
#@title Esempio immagini { vertical-output: true, display-mode: "form" }
numImages = 10 #@param {type:"integer"}
offsetTraining = 5000 #@param {type:"integer"}
offsetValidation = 50 #@param {type:"integer"}

if offsetTraining < 0:
  offsetTraining = np.random.randint(low = 0, high = len(trainingData) - numImages)
if offsetValidation < 0:
  offsetValidation = np.random.randint(low = 0, high = len(validationData) - numImages)

print("Offset training : ", offsetTraining)
print("Offset validation : ", offsetValidation)

figure = plt.figure(figsize = (30,30),constrained_layout=False)
gs = figure.add_gridspec(nrows=2, ncols=numImages, right = 0.4, left = 0.1,top = 0.2, bottom = 0.1)
count = 0
for i in range(numImages):
  ax = figure.add_subplot(gs[0,i])
  ax.imshow(trainingData[offsetTraining + count])
  ax.axis('off')
  ax.grid(False)
  count += 1
  


figure = plt.figure(figsize = (30,30),constrained_layout=False)
gs = figure.add_gridspec(nrows=2, ncols=numImages, right = 0.4, left = 0.1,top = 0.2, bottom = 0.1)
count = 0
for i in range(numImages):
  ax = figure.add_subplot(gs[0,i])
  ax.imshow(validationData[offsetValidation + count])
  ax.axis('off')
  ax.grid(False)
  count += 1

In [0]:
def BigResNet():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildConvBlock(inputs, layers = 3, size = 64, kernelSize = 3, poolSize = 2, poolStrides = 2, flatten = False)
    layer = buildResNetBlock(layer, 128, 128, 512)
    layer = buildResNetBlock(layer, 128, 128, 512)
    layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
    layer = buildResNetBlock(layer, 256, 256, 1024)
    layer = buildResNetBlock(layer, 256, 256, 1024)
    layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
    layer = buildResNetBlock(layer, 512, 512, 2048)
    outputs = buildGlobalSoftmax(layer, numberOfLabels = 5, kernelSize=3, denseDepth = 2, denseSize=128, dropout = 0.35)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure,"BigResNetGroupDetection")

In [0]:
models = Models()

trInfo = TrainingInfo.getDefaultTPU(
    trainingData,
    trainingLabels,
    validationData,
    validationLabels
)

trInfo.setParameters(
    learningRateList = [0.001],
    fineTuningIterations = 4,
    mainEpochs = 200,
    fineTuningEpochs = 50,
    batchSize = 128,
    validationFrequency = 10,
    validationFrequencyFT = 5,
    metrics = ['sparse_categorical_accuracy'],
    classWeights = None
)

generatedModel = BigResNet()
models.addModel(generatedModel[1], generatedModel[0], trInfo)

In [0]:
#@title Training ( versione 4) { vertical-output: true, display-mode: "form" }
cleanLastSession = False #@param {type:"boolean"}
if 'lastSession' in locals():
  if cleanLastSession == True:
    for trainedModel in lastSession:
      print("Rimuovo il training del modello", trainedModel)
      shutil.rmtree(trainedModel)
    lastSession = []
else:
  lastSession = []
#@markdown ---
verboseTraining = 0 #@param {type:"integer"}
#@markdown ---
mainTrainingEarlyStoppingDelta = 0.01 #@param {type:"number"}
mainTrainingEarlyStoppinPatience = 15 #@param {type:"integer"}
fineTuningEarlyStoppingDelta = 0.003 #@param {type:"number"}
fineTuningEarlyStoppinPatience = 10 #@param {type:"integer"}
#@markdown ---

trainingPath = rootPath + phase + "/"

dataToLog  = ["AUG : "+ str(useDataAugmentation)+"\n",
                    "TR_AUG : " + str(NUMBER_OF_TRAINING_AUG)+"\n",
                    "VAL_AUG : " + str(NUMBER_OF_VALIDATION_AUG)+"\n",
                    "brightness_range_training : " + str(brightness_range_training) + "\n",
                    "rotation_range_training : " + str(rotation_range_training) + "\n",
                    "width_shift_range_training : " + str(width_shift_range_training) + "\n",
                    "height_shift_range_training : " + str(height_shift_range_training) + "\n",
                    "shear_range_training : " + str(shear_range_training) + "\n",     
                    "zoom_range_training : " + str(zoom_range_training) + "\n",
                    "channel_shift_range_training : " + str(channel_shift_range_training) + "\n",
                    "brightness_range_validation : " + str(brightness_range_validation) + "\n",
                    "rotation_range_validation : " + str(rotation_range_validation) + "\n",
                    "width_shift_range_validation : " + str(width_shift_range_validation) + "\n",
                    "height_shift_range_validation : " + str(height_shift_range_validation) + "\n",
                    "shear_range_validation : " + str(shear_range_validation) + "\n",     
                    "zoom_range_validation : " + str(zoom_range_validation) + "\n",
                    "channel_shift_range_validation : " + str(channel_shift_range_validation) + "\n",
                    "sampling : " + str(samplingToLoad) + "\n",
                    "randomSeed : " + str(randomSeed)+"\n"]

if "useCustomLoad" in locals() and useCustomLoad == True:
  dataToLog.append("Custom_load : " + str(useCustomLoad)+"\n")
  dataToLog.append("Custom_ratio : " + str(customRatio) + "\n")  
        
train(tpu_address,
  trainingPath,
  models, 
  verboseTraining = 1,
  mainTrainingEarlyStoppingDelta = mainTrainingEarlyStoppingDelta,
  mainTrainingEarlyStoppinPatience = mainTrainingEarlyStoppinPatience,
  fineTuningEarlyStoppingDelta = fineTuningEarlyStoppingDelta,
  fineTuningEarlyStoppinPatience = fineTuningEarlyStoppinPatience,
  stringToLog = ''.join(dataToLog),
  lastSession = lastSession)
  
    
print("Fine training")

In [0]:
#@title Group test

testData, padding = loadTestData(testDir, width, height)
labels = np.load(testDir + "labels.npy")
groupLabels = np.zeros((len(labels)), dtype="int32")
print("Test data ( padding", padding,") :", len(testData))

bigResNetMdodel = BigResNet()[0]

predictions = testModels([
                           (
                              bigResNetMdodel,
                              [
                               rootPath + 'trainingPhaseGroupModel/BigResNetGroupDetection_2019_8_16_14_38_11/0.001/FineTuning/6.25e-05/weights/epoch_45_valLoss_0.0181.hdf5'
                              ]
                           )
                         ],
                         testData,
                         useTPU,
                         tpu_address
                       )  

savePredictions(predictions, tmpPath + "step1Predictions.npy")

for i in range(len(labels)):
  groupLabels[i] = labelsInfo[labels[i]][0]

confusionMatrix = predictionEvaluation(predictions, groupLabels, [0], topN = 5)
confusionMatrixStat(confusionMatrix[0], printStat = True)
plotConfusionMatrix(confusionMatrix[0], "TOP 1")

# ***Segue l'allenamento dei singoli modelli***

In [0]:
#@title Caricamento dei dati { vertical-output: true, display-mode: "form" }

if 'trainingData' in locals():
  del trainingData
if 'trainingLabels' in locals():
  del trainingLabels
if 'validationData' in locals():  
  del validationData
if 'validationLabels' in locals():
  del validationLabels

#@markdown Seed per la genezione di nuove immagini e la scelta casuale per custom
#@markdown load
samplingToLoad = 1#@param{type:'integer'}
randomSeed = 41117#@param{type:'integer'}

trainingDir = rootPath + 'data/sampling' + str(samplingToLoad) + '_' + str(width) + 'x' + str(height) + '/training/'
validationDir = rootPath + 'data/sampling' + str(samplingToLoad) + '_' + str(width) + 'x' + str(height) + '/validation/'

#riproducibilità dei training
resetSeed(15 + randomSeed)

#@markdown ---
# Load selettivo dei singoli gruppi
loadG0 = False #@param {type:"boolean"}
loadG1 = True #@param {type:"boolean"}
loadG2 = False #@param {type:"boolean"}
loadG3 = False #@param {type:"boolean"}
loadG4 = False #@param {type:"boolean"}

loadGroup = [loadG0, loadG1, loadG2, loadG3, loadG4]

#@markdown ---
useDataAugmentation = True #@param {type:"boolean"}
#@markdown Se selezionato ignora i valori assegnati ai gruppi sottostanti e usa 
#@markdown un valore perogni label invece che ogni gruppo. Sono definiti nel codice
usePerLabelDataAug = False #@param {type:"boolean"}
#Numero di immagini da generare per ogni immagine originale
NUMBER_OF_TRAINING_AUG_G0 = 7 #@param {type:"slider", min:0, max:20, step:1}
NUMBER_OF_VALIDATION_AUG_G0 = 2 #@param {type:"slider", min:0, max:5, step:1}
NUMBER_OF_TRAINING_AUG_G1 = 9 #@param {type:"slider", min:0, max:20, step:1}
NUMBER_OF_VALIDATION_AUG_G1 = 2 #@param {type:"slider", min:0, max:5, step:1}
NUMBER_OF_TRAINING_AUG_G2 = 7 #@param {type:"slider", min:0, max:20, step:1}
NUMBER_OF_VALIDATION_AUG_G2 = 4 #@param {type:"slider", min:0, max:5, step:1}
NUMBER_OF_TRAINING_AUG_G3 = 12 #@param {type:"slider", min:0, max:20, step:1}
NUMBER_OF_VALIDATION_AUG_G3 = 2 #@param {type:"slider", min:0, max:5, step:1}
NUMBER_OF_TRAINING_AUG_G4 = 27 #@param {type:"slider", min:0, max:30, step:1}
NUMBER_OF_VALIDATION_AUG_G4 = 5 #@param {type:"slider", min:0, max:10, step:1}

# da modificare se si vuole alterare la distribuzione delle classi
NUMBER_AUG_BY_LABEL = { 
                0 : ( 20 , 2 ), # group 0  label inside group 0
                1 : ( 1 , 2 ), # group 0  label inside group 1
                2 : ( 1 , 2 ), # group 0  label inside group 2
                3 : ( 2 , 2 ), # group 0  label inside group 3
                4 : ( 1 , 2 ), # group 0  label inside group 4
                5 : ( 1 , 2 ), # group 0  label inside group 5
                6 : ( 8 , 4 ), # group 4  label inside group 0
                7 : ( 2 , 2 ), # group 0  label inside group 6
                8 : ( 2 , 2 ), # group 0  label inside group 7
                9 : ( 2 , 2 ), # group 0  label inside group 8
                10 : ( 1 , 2 ), # group 0  label inside group 9
                11 : ( 2 , 2 ), # group 1  label inside group 0
                12 : ( 2 , 2 ), # group 3  label inside group 0
                13 : ( 2 , 2 ), # group 3  label inside group 1
                14 : ( 8 , 2 ), # group 3  label inside group 2
                15 : ( 6 , 2 ), # group 0  label inside group 10
                16 : ( 8 , 2 ), # group 0  label inside group 11
                17 : ( 5 , 2 ), # group 3  label inside group 3
                18 : ( 2 , 2 ), # group 1  label inside group 1
                19 : ( 13 , 2 ), # group 1  label inside group 2
                20 : ( 9 , 2 ), # group 1  label inside group 3
                21 : ( 10 , 2 ), # group 1  label inside group 4
                22 : ( 8 , 2 ), # group 1  label inside group 5
                23 : ( 6 , 2 ), # group 1  label inside group 6
                24 : ( 9 , 2 ), # group 1  label inside group 7
                25 : ( 1 , 2 ), # group 1  label inside group 8
                26 : ( 5 , 2 ), # group 1  label inside group 9
                27 : ( 15 , 5 ), # group 1  label inside group 10
                28 : ( 6 , 2 ), # group 1  label inside group 11
                29 : ( 10 , 2 ), # group 1  label inside group 12
                30 : ( 7 , 2 ), # group 1  label inside group 13
                31 : ( 3 , 2 ), # group 1  label inside group 14
                32 : ( 9 , 6 ), # group 4  label inside group 1
                33 : ( 4 , 2 ), # group 2  label inside group 0
                34 : ( 6 , 2 ), # group 2  label inside group 1
                35 : ( 3 , 2 ), # group 2  label inside group 2
                36 : ( 7 , 3 ), # group 2  label inside group 3
                37 : (11 , 5 ), # group 2  label inside group 4
                38 : ( 2 , 1 ), # group 2  label inside group 5
                39 : ( 7 , 4 ), # group 2  label inside group 6
                40 : ( 7, 4 ), # group 2  label inside group 7
                41 : ( 9 , 6 ), # group 4  label inside group 2
                42 : ( 9 , 6 ), # group 4  label inside group 3
              }

NUMBER_AUG_BY_GROUP = [
        (NUMBER_OF_TRAINING_AUG_G0, NUMBER_OF_VALIDATION_AUG_G0),
        (NUMBER_OF_TRAINING_AUG_G1, NUMBER_OF_VALIDATION_AUG_G1),
        (NUMBER_OF_TRAINING_AUG_G2, NUMBER_OF_VALIDATION_AUG_G2),
        (NUMBER_OF_TRAINING_AUG_G3, NUMBER_OF_VALIDATION_AUG_G3),
        (NUMBER_OF_TRAINING_AUG_G4, NUMBER_OF_VALIDATION_AUG_G4)
        ]
                  
NUMBER_OF_TRAINING_AUG = [0 for i in range(43)]
NUMBER_OF_VALIDATION_AUG = [0 for i in range(43)]

if useDataAugmentation == True:
  for key in labelsInfo.keys():
    NUMBER_OF_TRAINING_AUG[key] = NUMBER_AUG_BY_LABEL[key][0] if usePerLabelDataAug else NUMBER_AUG_BY_GROUP[labelsInfo[key][0]][0]
    NUMBER_OF_VALIDATION_AUG[key] = NUMBER_AUG_BY_LABEL[key][1] if usePerLabelDataAug else NUMBER_AUG_BY_GROUP[labelsInfo[key][0]][1]

print("Tr AUG : ", NUMBER_OF_TRAINING_AUG)
print("Val AUG : ", NUMBER_OF_VALIDATION_AUG)

# parametri generatore training per gruppo
brightness_range_training = [
    (0.25,0.6), (0.28,0.57), (0.2,0.7), (0.25,0.6), (0.7,1.0)
]
rotation_range_training = [25, 20, 10, 25,20]
width_shift_range_training=[8,7,10,8,10]
height_shift_range_training=[8,7,10,8,10]
shear_range_training=[15,12,10,15,12]
zoom_range_training=[(0.4,1.15),(0.45,1.15),(0.3,1.25),(0.4,1.15),(0.45,1.15)]
channel_shift_range_training=[0.13,0.10,0.15,0.13,0.15]

# parametri generatore validation per gruppo
brightness_range_validation = [
    (0.25,0.6), (0.28,0.57), (0.2,0.7), (0.25,0.6), (0.35,0.6)
]
rotation_range_validation = [25, 22, 15, 25, 30]
width_shift_range_validation=[8,8,9,8,9]
height_shift_range_validation=[8,8,9,8,9]
shear_range_validation=[15,12,12,15,15]
zoom_range_validation=[(0.4,1.15),(0.45,1.15),(0.25,1.3),(0.4,1.15),(0.35,1.15)]
channel_shift_range_validation=[0.13,0.115,0.16,0.13,0.15]

generatorTraining = []
generatorValidation= []

for group in range(len(groupInfo.keys())):
  #generatore utilizzato per le immagni di training
  generatorTraining.append(keras.preprocessing.image.ImageDataGenerator(
      brightness_range = brightness_range_training[group],
      rotation_range=rotation_range_training[group],
      width_shift_range=width_shift_range_training[group],
      height_shift_range=height_shift_range_training[group],
      shear_range=shear_range_training[group],
      zoom_range=zoom_range_training[group],
      channel_shift_range=channel_shift_range_training[group]))

  #generatore utilizzato per le immagini di validation
  generatorValidation.append(keras.preprocessing.image.ImageDataGenerator(
      brightness_range = brightness_range_validation[group],
      rotation_range=rotation_range_validation[group],
      width_shift_range=width_shift_range_validation[group],
      height_shift_range=height_shift_range_validation[group],
      shear_range=shear_range_validation[group],
      zoom_range=zoom_range_validation[group],
      channel_shift_range=channel_shift_range_validation[group]))

# numero di immagini originali disponibili per ogni classe
trainingClassCount = {}
#numero di immagini di training totali suddivide per gruppo
totalTrainingCount = [0 for group in groupInfo.keys()]
loadLabel = [False for i in range(43)]
# recupero il numero di immagini di training disponibili per ogni classe
files = list(filter(lambda fn : fn.startswith('resized'), os.listdir(trainingDir)))
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  numFilePath = trainingDir + 'num' + str(classLabel)
  with open(numFilePath, 'r') as file:
    originalNum = int(file.readline())
    trainingClassCount[classLabel] = originalNum
    groupIndex = labelsInfo[classLabel][0]
    if loadGroup[groupIndex] == True:
      loadLabel[classLabel] = True
      totalTrainingCount[groupIndex] += originalNum * (NUMBER_OF_TRAINING_AUG[classLabel] + 1)

# se uso le TPU in tf 1.13 il numero di immagini deve essere divisibile per 8
trainingPadEnd = [False for group in groupInfo.keys()]
trainingToPad = [0 for group in groupInfo.keys()]
if useTPU == True and tfVersion()['MAJOR'] == 1 and tfVersion()['MAJOR'] <= 13:
  for groupIndex in range(len(groupInfo.keys())):
    if loadGroup[groupIndex] and totalTrainingCount[groupIndex] % 8 != 0:
      # se uso le TPU la lunghezza dei dati deve essere divisibile per 8
      trainingToPad[groupIndex] = 8 - (totalTrainingCount[groupIndex]  % 8)
      totalTrainingCount[groupIndex] = totalTrainingCount[groupIndex]  + (trainingToPad[groupIndex])
      trainingPadEnd[groupIndex] = True
  
#preparo gli array che contengono le img di training
trainingData = []
trainingLabels = []
originalTrainingLabels = []

for groupIndex in range(len(groupInfo.keys())):
  trainingData.append(np.empty((totalTrainingCount[groupIndex], width, height, 3), dtype="float32"))
  trainingLabels.append( np.empty((totalTrainingCount[groupIndex]), dtype='int32'))
  originalTrainingLabels.append( np.empty((totalTrainingCount[groupIndex]), dtype='int32'))

# numero di immagini originali disponibili per ogni classe
validationClassCount = {}
#leggo quante img ci sono per ogni class di validation
totalValidationCount = [0 for group in groupInfo.keys()]


# recupero il numero di immagini di validation disponibili per ogni classe
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  numFilePath = validationDir + 'num' + str(classLabel)
  with open(numFilePath, 'r') as file:
    numFile = int(file.readline())
    validationClassCount[classLabel] = numFile
    groupIndex = labelsInfo[classLabel][0]
    if loadGroup[groupIndex] == True:
      totalValidationCount[groupIndex] += numFile * (NUMBER_OF_VALIDATION_AUG[classLabel]+ 1)

# se uso le TPU in tf 1.13 il numero di immagini deve essere divisibile per 8
validationPadEnd = [False for group in groupInfo.keys()]
validationToPad = [0 for group in groupInfo.keys()]
if tfVersion()['MAJOR'] == 1 and tfVersion()['MAJOR'] <= 13:
  for groupIndex in range(len(groupInfo.keys())):
    if totalValidationCount[groupIndex] % 8 != 0:
      validationToPad[groupIndex] = 8 - (totalValidationCount[groupIndex]  % 8)
      totalValidationCount[groupIndex] = totalValidationCount[groupIndex]  + (validationToPad[groupIndex])
      validationPadEnd[groupIndex] = True
  
#preparo gli array che contengono le img di validation
validationData = []
validationLabels = []
originalValidationLabels = []
for groupIndex in range(len(groupInfo.keys())):
  validationData.append(np.empty((totalValidationCount[groupIndex], width, height, 3), dtype="float32"))
  validationLabels.append( np.empty((totalValidationCount[groupIndex]), dtype='int32'))
  originalValidationLabels.append( np.empty((totalValidationCount[groupIndex]), dtype='int32'))

#numero di immagini elaborate per gruppo
validationCount = [0 for group in groupInfo.keys()]
trainingCount = [0 for group in groupInfo.keys()]
loadedClass = 0

#caricamento dati
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  if loadLabel[classLabel] == True:
    #carico le img di training originali
    trainingImages = np.memmap(trainingDir + fileName, dtype=np.uint8,
                mode='r', shape=(trainingClassCount[classLabel],width,height,3))

    group = labelsInfo[classLabel][0]

    for img in trainingImages:
      trainingData[group][trainingCount[group]]  = img/255.0
      trainingLabels[group][trainingCount[group]] = labelsInfo[classLabel][1]
      originalTrainingLabels[group][trainingCount[group]] = classLabel
      trainingCount[group] += 1

    #carico le immagini di validation
    validationImages = np.memmap(validationDir + fileName, dtype=np.uint8,
                mode='r', shape=(validationClassCount[classLabel],width,height,3))

    for img in validationImages:
      validationData[group][validationCount[group]] = img/255.0
      validationLabels[group][validationCount[group]] = labelsInfo[classLabel][1]
      originalValidationLabels[group][validationCount[group]] = classLabel
      validationCount[group] += 1

  loadedClass += 1
  if (loadedClass) % 5 == 0 or loadedClass == len(files):
    print('Caricamento file ', loadedClass, 'su', len(files))

# fine for caricamento classi

# per ogni immagine genero delle nuove immagini con trasformazioni casuali
print("\nGenero le immagini aggiuntive necessarie:\n")
resetSeed(48560  + randomSeed)
for group in range(len(groupInfo.keys())):
  for imageIndex in range(0, trainingCount[group]):
    #print(group, imageIndex, originalTrainingLabels[group][imageIndex], NUMBER_OF_TRAINING_AUG[originalTrainingLabels[group][imageIndex]])    
    for augIndex in range(0, NUMBER_OF_TRAINING_AUG[originalTrainingLabels[group][imageIndex]]):
      trainingData[group][trainingCount[group]] = generateRandomImage(generatorTraining[group], trainingData[group][imageIndex])/255.0
      trainingLabels[group][trainingCount[group]] = trainingLabels[group][imageIndex]
      trainingCount[group] += 1
  print("Gruppo", group, "training terminato")

resetSeed(-420360  + randomSeed)
for group in range(len(groupInfo.keys())):
  for imageIndex in range(0, validationCount[group]):
    for augIndex in range(0, NUMBER_OF_VALIDATION_AUG[originalValidationLabels[group][imageIndex]]):
      validationData[group][validationCount[group]] = generateRandomImage(generatorValidation[group], validationData[group][imageIndex])/255.0
      validationLabels[group][validationCount[group]] = validationLabels[group][imageIndex]
      validationCount[group] += 1
  print("Gruppo", group, "validation terminato")

# Se necessario copio l'ultima immagine per rendere la lunghezza divisibile per 8
for group in range(len(groupInfo.keys())):       
  if trainingPadEnd[group] == True:
    for index in range(1, trainingToPad[group] + 1):
      trainingData[group][-index] = trainingData[group][len(trainingData[group]) - (trainingToPad[group] + 1)]
      trainingLabels[group][-index] = trainingLabels[group][len(trainingLabels[group]) - (trainingToPad[group] + 1)]
    
  if validationPadEnd[group] == True:
    for index in range(1, validationToPad[group] + 1):
      validationData[group][-index] = validationData[group][len(validationData[group]) - (validationToPad[group] + 1)]
      validationLabels[group][-index] = validationLabels[group][len(validationLabels[group]) - (validationToPad[group] + 1)]
    
print("Fine padding")      

# statistiche sulla distribuzione delle labels      

for index in range(5):
  if loadGroup[index] == True:
    plt.figure(figsize=(10,5))
    maxLabel = max(trainingLabels[index])+1
    if maxLabel > 43:
      raise RuntimeError("C'è un errore grave nel codice, ricontrolla.")
    plt.hist(trainingLabels[index], alpha = 0.5, bins=maxLabel, label = "training")
    plt.hist(validationLabels[index], alpha = 0.5, bins=maxLabel, label = "validation")
    plt.xticks(np.arange(0, maxLabel, 1))
    plt.xlim(left=0, right=maxLabel)
    plt.title('Group' + str(index))
    plt.legend(loc='upper right')
    plt.show()


In [0]:
#@title Esempio immagini { vertical-output: true, display-mode: "form" }
group = 4 #@param {type:"integer"}
numImages = 10 #@param {type:"integer"}
offsetTraining = 5000 #@param {type:"integer"}
offsetValidation = 2000 #@param {type:"integer"}

if offsetTraining < 0:
  offsetTraining = np.random.randint(low = 0, high = len(trainingData[group]) - numImages)
if offsetValidation < 0:
  offsetValidation = np.random.randint(low = 0, high = len(validationData[group]) - numImages)

print("Offset training : ", offsetTraining)
print("Offset validation : ", offsetValidation)

figure = plt.figure(figsize = (30,30),constrained_layout=False)
gs = figure.add_gridspec(nrows=2, ncols=numImages, right = 0.4, left = 0.1,top = 0.2, bottom = 0.1)
count = 0
for i in range(numImages):
  ax = figure.add_subplot(gs[0,i])
  ax.imshow(trainingData[group][offsetTraining + count])
  ax.axis('off')
  ax.grid(False)
  count += 1
  


figure = plt.figure(figsize = (30,30),constrained_layout=False)
gs = figure.add_gridspec(nrows=2, ncols=numImages, right = 0.4, left = 0.1,top = 0.2, bottom = 0.1)
count = 0
for i in range(numImages):
  ax = figure.add_subplot(gs[0,i])
  ax.imshow(validationData[group][offsetValidation + count])
  ax.axis('off')
  ax.grid(False)
  count += 1

In [0]:
def model0_0(numberOfLabels):
  
  f1_1x1 = 32
  f1_3x3 = 128
  f1_5x5 = 64
  f1_maxPool = 32

  f2_1x1 = 64
  f2_3x3 = 256
  f2_5x5 = 128
  f2_maxPool = 64

  f3_1x1 = 128
  f3_3x3 = 512
  f3_5x5 = 192
  f3_maxPool = 128

  def closure():
      inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
      layer = buildConvBlock(inputs, layers = 3, size = 64, kernelSize = 5, poolSize = 2, poolStrides = 2, flatten = False)
      layer = buildResidualInceptionBlockV4(layer, layer, f1_1x1, f1_3x3, f1_5x5, f1_maxPool)
      layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
      layer = buildResidualInceptionBlockV4(layer, layer, f2_1x1, f2_3x3, f2_5x5, f2_maxPool, 384, False)
      layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
      layer = buildResidualInceptionBlockV4(layer, layer, f3_1x1, f1_3x3, f3_5x5, f3_maxPool)
      layer = buildDenseLayer(layer, 2,64,0.0015,True,True,True,0.3)
      outputs = buildDenseSoftmax(layer, numberOfLabels=numberOfLabels, flattenInput=False)
      return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "MODEL0_0_INC")

def model1_3(numberOfLabels):
  
  f1_1x1 = 32
  f1_3x3 = 128
  f1_5x5 = 64
  f1_maxPool = 32

  f2_1x1 = 64
  f2_3x3 = 256
  f2_5x5 = 128
  f2_maxPool = 64

  f3_1x1 = 128
  f3_3x3 = 512
  f3_5x5 = 192
  f3_maxPool = 128

  def closure():
      inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
      layer = buildConvBlock(inputs, layers = 3, size = 64, kernelSize = 5, poolSize = 2, poolStrides = 2, flatten = False)
      layer = buildResidualInceptionBlockV4(layer, layer, f1_1x1, f1_3x3, f1_5x5, f1_maxPool)
      layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
      layer = buildResidualInceptionBlockV4(layer, layer, f2_1x1, f2_3x3, f2_5x5, f2_maxPool, 384, False)
      layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
      layer = buildResidualInceptionBlockV4(layer, layer, f3_1x1, f1_3x3, f3_5x5, f3_maxPool)
      layer = buildDenseLayer(layer, 2,64,0.0015,True,True,True,0.3)
      outputs = buildDenseSoftmax(layer, numberOfLabels=numberOfLabels, flattenInput=False)
      return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "MODEL1_3_INC")


def model2_1(numberOfLabels):
  
  f1_1x1 = 32
  f1_3x3 = 128
  f1_5x5 = 64
  f1_maxPool = 32

  f2_1x1 = 64
  f2_3x3 = 256
  f2_5x5 = 128
  f2_maxPool = 64

  f3_1x1 = 128
  f3_3x3 = 512
  f3_5x5 = 192
  f3_maxPool = 128

  def closure():
      inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
      layer = buildConvBlock(inputs, layers = 3, size = 64, kernelSize = 5, poolSize = 2, poolStrides = 2, flatten = False)
      layer = buildResidualInceptionBlockV4(layer, layer, f1_1x1, f1_3x3, f1_5x5, f1_maxPool)
      layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
      layer = buildResidualInceptionBlockV4(layer, layer, f2_1x1, f2_3x3, f2_5x5, f2_maxPool, 384, False)
      layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
      layer = buildResidualInceptionBlockV4(layer, layer, f3_1x1, f1_3x3, f3_5x5, f3_maxPool)
      layer = buildDenseLayer(layer, 2,64,0.0015,True,True,True,0.3)
      outputs = buildDenseSoftmax(layer, numberOfLabels=numberOfLabels, flattenInput=False)
      return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "INC_MODEL2_1")


def model3_0(numberOfLabels):
  MULT = 2
  L1_B = 128
  L1_A = L1_B * MULT
  L2_B = 192
  L2_A = L1_B * MULT
  L3_B = 256
  L3_A = L1_B * MULT
  
  def closure():
      inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
      layer = buildConvBlock(inputs, layers = 2, size = 64, kernelSize = 3, poolSize = 3, poolStrides = 2, flatten = False)
      layer = buildResNetBlock(layer, L1_B, L1_B, L1_A)
      layer = buildResNetBlock(layer, L1_B, L1_B, L1_A)
      layer = buildResNetBlock(layer, L1_B, L1_B, L1_A)
      layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
      layer = buildResNetBlock(layer, L2_B, L2_B, L2_A)
      layer = buildResNetBlock(layer, L2_B, L2_B, L2_A)
      layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
      layer = buildResNetBlock(layer, L3_B, L3_B, L3_A)
      layer = buildResNetBlock(layer, L3_B, L3_B, L3_A)
      layer = buildDenseLayer(layer, 2, 64, regularizers = 0.005, useBatch = True, flattenInput = True, leaky = True, dropout = 0.2)
      outputs = buildDenseSoftmax(layer, numberOfLabels = numberOfLabels, flattenInput = False)
      return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "MODEL3_0_LEAKY")


def model4_6(numberOfLabels):
  
  f1_1x1 = 32
  f1_3x3 = 128
  f1_5x5 = 64
  f1_maxPool = 32

  def closure():
      inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
      layer = buildConvBlock(inputs, layers = 3, size = 64, kernelSize = 5, poolSize = 2, poolStrides = 2, flatten = False)
      layer = buildResidualInceptionBlockV4(layer, layer, f1_1x1, f1_3x3, f1_5x5, f1_maxPool)
      layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
      layer = buildResidualInceptionBlockV4(layer, layer, f1_1x1, f1_3x3, f1_5x5, f1_maxPool, 384, False)
      layer = buildMaxPoolLayer(layer, poolStrides = 2, poolSize = 2)
      layer = buildResidualInceptionBlockV4(layer, layer, f1_1x1, f1_3x3, f1_5x5, f1_maxPool)# 392, False)
      layer = buildDenseLayer(layer, 2,64,0.0015,True,True,True,0.3)
      outputs = buildDenseSoftmax(layer, numberOfLabels=numberOfLabels, flattenInput=False)
      #outputs = buildGlobalSoftmax(layer, numberOfLabels = numberOfLabels, denseSize = 64, denseDepth = 2, kernelSize = 3)
      return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "INC_MODEL4_6")

In [0]:
subGroupModels = Models()

for group in [1]:#range(len(groupInfo.keys())):
  if loadGroup[group] == False:
    raise RuntimeError("Non hai caricato i dati per il gruppo", group)
  trInfo = TrainingInfo.getDefaultTPU(
      trainingData[group],
      trainingLabels[group],
      validationData[group],
      validationLabels[group]
  )
  
  uniqueLables, counts = np.unique(trainingLabels[group], return_counts=True)
  classWeights = dict(zip(uniqueLables, [math.sqrt(math.sqrt(e)) for e in max(counts) / counts]))
  print(classWeights)
  
  trInfo.setParameters(
      learningRateList = [0.0005],
      fineTuningIterations = 3,
      mainEpochs = 150,
      fineTuningEpochs = 60,
      batchSize = 128,
      validationFrequency = 1,
      validationFrequencyFT = 1,
      metrics = ['sparse_categorical_accuracy'],
      classWeights = classWeights#None # Non supportato dalle TPU nella versione 1.14, utilizzare None
  )
  generatedModel = model1_3(max(trainingLabels[group]) + 1)
  subGroupModels.addModel(generatedModel[1] + '_group' + str(group), generatedModel[0], trInfo)

In [0]:
#@title Training sub-group (versione 2) { vertical-output: true, display-mode: "form" }
cleanLastSession = False #@param {type:"boolean"}
if 'lastSession' in locals():
  if cleanLastSession == True:
    for trainedModel in lastSession:
      print("Rimuovo il training del modello", trainedModel)
      shutil.rmtree(trainedModel)
    lastSession = []
else:
  lastSession = []
#@markdown ---
verboseTraining = 0 #@param {type:"integer"}
#@markdown ---
mainTrainingEarlyStoppingDelta = 0.01 #@param {type:"number"}
mainTrainingEarlyStoppinPatience = 15 #@param {type:"integer"}
fineTuningEarlyStoppingDelta = 0.003 #@param {type:"number"}
fineTuningEarlyStoppinPatience = 10 #@param {type:"integer"}
#@markdown ---

trainingPath = rootPath + trainingOutputDir + "/"

dataToLog  = ["AUG : "+ str(useDataAugmentation)+"\n",
                    "TR_AUG : " + str(NUMBER_OF_TRAINING_AUG)+"\n",
                    "VAL_AUG : " + str(NUMBER_OF_VALIDATION_AUG)+"\n",
                    "brightness_range_training : " + str(brightness_range_training) + "\n",
                    "rotation_range_training : " + str(rotation_range_training) + "\n",
                    "width_shift_range_training : " + str(width_shift_range_training) + "\n",
                    "height_shift_range_training : " + str(height_shift_range_training) + "\n",
                    "shear_range_training : " + str(shear_range_training) + "\n",     
                    "zoom_range_training : " + str(zoom_range_training) + "\n",
                    "channel_shift_range_training : " + str(channel_shift_range_training) + "\n",
                    "brightness_range_validation : " + str(brightness_range_validation) + "\n",
                    "rotation_range_validation : " + str(rotation_range_validation) + "\n",
                    "width_shift_range_validation : " + str(width_shift_range_validation) + "\n",
                    "height_shift_range_validation : " + str(height_shift_range_validation) + "\n",
                    "shear_range_validation : " + str(shear_range_validation) + "\n",     
                    "zoom_range_validation : " + str(zoom_range_validation) + "\n",
                    "channel_shift_range_validation : " + str(channel_shift_range_validation) + "\n",
                    "sampling : " + str(samplingToLoad) + "\n",
                    "randomSeed : " + str(randomSeed)+"\n"]

if "useCustomLoad" in locals() and useCustomLoad == True:
  dataToLog.append("Custom_load : " + str(useCustomLoad)+"\n")
  dataToLog.append("Custom_ratio : " + str(customRatio) + "\n")  
        
history = train(tpu_address,
  trainingPath,
  subGroupModels, 
  verboseTraining = 1,
  mainTrainingEarlyStoppingDelta = mainTrainingEarlyStoppingDelta,
  mainTrainingEarlyStoppinPatience = mainTrainingEarlyStoppinPatience,
  fineTuningEarlyStoppingDelta = fineTuningEarlyStoppingDelta,
  fineTuningEarlyStoppinPatience = fineTuningEarlyStoppinPatience,
  stringToLog = ''.join(dataToLog),
  lastSession = lastSession)
  
    
print("Fine training")

In [0]:
plotCM = False #@param {type:"boolean"}
loadG0 = True #@param {type:"boolean"}
loadG1 = True #@param {type:"boolean"}
loadG2 = True #@param {type:"boolean"}
loadG3 = True #@param {type:"boolean"}
loadG4 = True #@param {type:"boolean"}

loadGroup = [loadG0, loadG1, loadG2, loadG3, loadG4]

if 'testData' not in locals():
  testData, padding = loadTestData(testDir, width, height, padding = False)
if 'predictions' not in locals():
  predictions = loadPredictions(tmpPath + "step1Predictions.npy")
if 'trueLabels' not in locals():
  trueLabels = np.load(testDir + "labels.npy")

# Tutte le inferenze del primo blocco vengono suddivise per gruppo per poi
# essere passate al classificatore specifico, le inferenze dovute al padding
# vengono ignorate
predictedGroupLabels = np.zeros((len(trueLabels)),dtype="int32")

for predictionIndex in range(len(trueLabels)):
  # sommo il contributo di tutti i modelli che hanno generato un'inferenze
  prediction = np.zeros((5), dtype="float")
  for modelIndex in range(len(predictions)):
    for weightsIndex in range(len(predictions[0])):
      prediction = np.add(prediction, predictions[modelIndex][weightsIndex][predictionIndex])
  predictedGroupLabels[predictionIndex] = np.argmax(prediction)

# Divido le label originali (con valore da 0 a 42) per gruppo, utilizzando il gruppo 
# (eventualmente sbagliato) derivante dall'inferenza dello step 1 e non il gruppo reale (corretto)
# calcolato sulla label originale
trueLabelsByPredictedGroup = [[],[],[],[],[]]
trueLabelsByRealGroup = [[],[],[],[],[]]
testDataGroup = [[],[],[],[],[]]
for index in range(len(predictedGroupLabels)):
  testDataGroup[predictedGroupLabels[index]].append(testData[index])
  trueLabelsByPredictedGroup[predictedGroupLabels[index]].append(trueLabels[index]) 
  trueLabelsByRealGroup[labelsInfo[trueLabels[index]][0]].append(trueLabels[index])

# Contiene degli array con le inferenze per ogni gruppo
result = []  
# Numero di label uniche per ogni gruppo
maxLabels = [12,15,8,4,4]

count = 0

## 
# IMPOSTARE QUI I PESI DA UTILIZZARE PER I 5 MODELLI
##
weights = [
  # grouppo 0
    [
     #5656
     rootPath + 'trainingPhaseGroupModel/MODEL0_0_INC_group0_2019_8_23_14_57_16/0.0005/Main/weights/epoch_31_valLoss_0.0504.hdf5',
    ],
    
  # grouppo 1
    [
     # gruppo 2764
     #2743
     rootPath+'trainingPhaseGroupModel/MODEL1_3_INC_group1_2019_8_24_19_15_53/0.0005/FineTuning/0.000125/weights/epoch_15_valLoss_0.0142.hdf5',
     #2756
     rootPath + 'trainingPhaseGroupModel/MODEL1_3_INC_group1_2019_8_24_14_48_42/0.0005/FineTuning/6.25e-05/weights/epoch_14_valLoss_0.0188.hdf5'
    ],
  # gruppo 2
    [
      #1757 e 1756 fanno 1762
      #1757         
      rootPath + 'trainingPhaseGroupModel/INC_MODEL2_1_group2_2019_8_23_6_29_18/0.0005/FineTuning/0.000125/weights/epoch_05_valLoss_0.0416.hdf5',
      #1756         
      rootPath  + 'trainingPhaseGroupModel/INC_MODEL2_1_group2_2019_8_22_15_29_53/0.0005/Main/weights/epoch_39_valLoss_0.0662.hdf5'
    ],
    
  # grouppo 3
    [
      #2024
      rootPath + 'trainingPhaseGroupModel/MODEL3_0_LEAKY_group3_2019_8_15_20_49_33/0.0005/FineTuning/6.25e-05/weights/epoch_12_valLoss_0.0110.hdf5'
    ],
    
  # grouppo 4
    [
     #358
     rootPath + 'trainingPhaseGroupModel/INC_MODEL4_6_group4_2019_8_22_6_26_47/0.0005/FineTuning/0.000125/weights/epoch_03_valLoss_0.0537.hdf5'
    ]
]

for tdGroup in testDataGroup:
  if loadGroup[count]:
    print("Gruppo", count, "caricato.")
    result.append(loadPredictions(tmpPath + "step2PredictionsG" + str(count) + ".npy"))
    count += 1
    continue

  if len(tdGroup) % 8 != 0:
    for index in range(len(tdGroup),len(tdGroup) + (8-len(tdGroup) % 8)):
      tdGroup.append(tdGroup[-1])

  data = np.asarray(tdGroup, dtype=np.float32)

  ##
  # IMPOSTARE QUI I MODELLI DA UTILIZZARE
  ##
  modelList = [ 
               model0_0(maxLabels[count])[0],
               model1_3(maxLabels[count])[0],
               model2_1(maxLabels[count])[0],
               model3_0(maxLabels[count])[0],
               model4_6(maxLabels[count])[0]
  ]
    
  result.append(testModels(
                    [
                      (modelList[count], weights[count])
                    ],
                   data,
                   useTPU,
                   tpu_address
                   )
      )
  savePredictions(result[-1], tmpPath + "step2PredictionsG" + str(count) + ".npy")
  count += 1

print("")


globalConfusionMatrix = [[0 for i in range(43)] for i in range(43)]
groupConfusionMatrix = []

for index in range(len(result)):
  cm = predictionEvaluation(result[index], trueLabelsByPredictedGroup[index], [0], topN = 1, lookUpTable=lookUpTable[index])
  groupConfusionMatrix.append(cm[0])
  correct, total = confusionMatrixStat(cm[0], printStat = False)
  print("Corrette", correct, "        su", total, "{0:.3f}".format((float(correct)/total * 100)), "% (predicted)", "    su : " , len(trueLabelsByRealGroup[index]), ":",    "{0:.3f}".format((float(correct)/len(trueLabelsByRealGroup[index]) * 100)), "% (reali)")
  for r in range(len(cm[0])):
    for c in range(len(cm[0])):
      globalConfusionMatrix[r][c] += cm[0][r][c]
  if plotCM:
    plotConfusionMatrix(cm[0], title = "group" + str(index))
print("")
print("Globale")
correct, total = confusionMatrixStat(globalConfusionMatrix, printStat = True)

```
Corrette 5656         su 5672 99.718 % (predicted)     su :  5670 : 99.753 % (reali)
Corrette 2764         su 2790 99.068 % (predicted)     su :  2790 : 99.068 % (reali)
Corrette 1762         su 1771 99.492 % (predicted)     su :  1770 : 99.548 % (reali)
Corrette 2024         su 2038 99.313 % (predicted)     su :  2040 : 99.216 % (reali)
Corrette 358         su 359 99.721 % (predicted)     su :  360 : 99.444 % (reali)

Globale
Corrette 12564 su 12630 : 99.477 %
```